#### We are trying to modify the power plants of this grid to obtain this energy mix:
<img src="images/target_em.png" width="30%" height="30%">

In [1]:
import numpy as np
import os
import pandas as pd
import grid2op
from grid2op.Chronics import ChangeNothing
from chronix2grid.kpi.Generator_parameter_checker import EnergyMix_AprioriChecker

#### We load the original prods_charac.csv

In [2]:
input_path = os.path.join("..", "example", "input", "generation", "case118_l2rpn_wcci")
df = pd.read_csv(os.path.join(input_path, "prods_charac.csv"))

avg_pmaxs = df.groupby(["type"])["Pmax"].mean()
types = avg_pmaxs.index.to_numpy()
avg_pmaxs = avg_pmaxs.to_numpy()

In [3]:
df

,name,Pmax,Pmin,type,bus,max_ramp_up,max_ramp_down,min_up_time,min_down_time,marginal_cost,shut_down_cost,start_cost,zone,generator,x,y,V
0,gen_3_0,33.6,0.0,wind,3,NaN,NaN,0.0,0.0,0.0,0,0.0,R1,0,62,-101,142.1
1,gen_3_1,33.6,0.0,wind,3,NaN,NaN,0.0,0.0,0.0,0,0.0,R1,70,62,-101,142.1
2,gen_7_2,400.0,0.0,nuclear,7,2.8,2.8,2016.0,2016.0,35.0,0,27728.7,R1,2,74,-293,355.4
3,gen_9_3,300.0,0.0,thermal,9,5.8,5.8,12.0,12.0,50.0,0,3392.3,R1,3,74,-390,355.4
4,gen_10_4,37.3,0.0,solar,10,NaN,NaN,0.0,0.0,0.0,0,0.0,R1,69,116,-104,142.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,gen_106_57,67.2,0.0,wind,106,NaN,NaN,0.0,0.0,0.0,0,0.0,R3,48,988,-582,142.1
58,gen_110_58,150.0,0.0,thermal,110,4.3,4.3,24.0,24.0,50.0,0,15319.4,R3,50,900,-707,142.1
59,gen_111_59,67.2,0.0,wind,111,NaN,NaN,0.0,0.0,0.0,0,0.0,R3,51,987,-699,142.1
60,gen_112_60,100.0,0.0,thermal,112,2.8,2.8,24.0,24.0,50.0,0,18016.4,R1,52,203,-276,142.1


In [21]:
df["type"].value_counts()

wind       49
hydro       6
nuclear     3
solar       3
thermal     1
Name: type, dtype: int64

In [4]:
def get_info_by_type(info):
  res = []
  for t in types:
    res.append(df[df["type"] == t].iloc[0][info])
  return np.array(res)

#### We recover some informations about each power plant type

In [5]:
df = pd.read_csv(os.path.join(input_path, "prods_charac.csv"))


n = df.shape[0]
pmaxs = [250.0, 400.0, 74.7, 200.0, 67.2]
max_ramp_up = get_info_by_type("max_ramp_up")
max_ramp_down = get_info_by_type("max_ramp_down")
min_up_time = get_info_by_type("min_up_time")
min_down_time = get_info_by_type("min_down_time")
marginal_cost = get_info_by_type("marginal_cost")
shut_down_cost = get_info_by_type("shut_down_cost")
start_cost = get_info_by_type("start_cost")

#### We load the solution returned by the solver and modify the original csv according to the solution

In [6]:
file = open("optimization/solver/build/result.txt")
for i in range(n):
  idx = int(file.readline())
  df.at[i, "type"] = types[idx]
  df.at[i, "Pmax"] = pmaxs[idx]
  df.at[i, "max_ramp_up"] = max_ramp_up[idx]
  df.at[i, "max_ramp_down"] = max_ramp_down[idx]
  df.at[i, "min_up_time"] = min_up_time[idx]
  df.at[i, "min_down_time"] = min_down_time[idx]
  df.at[i, "marginal_cost"] = marginal_cost[idx]
  df.at[i, "shut_down_cost"] = shut_down_cost[idx]
  df.at[i, "start_cost"] = start_cost[idx]

In [7]:
df

,name,Pmax,Pmin,type,bus,max_ramp_up,max_ramp_down,min_up_time,min_down_time,marginal_cost,shut_down_cost,start_cost,zone,generator,x,y,V
0,gen_3_0,67.2,0.0,wind,3,NaN,NaN,0.0,0.0,0.0,0,0.0,R1,0,62,-101,142.1
1,gen_3_1,67.2,0.0,wind,3,NaN,NaN,0.0,0.0,0.0,0,0.0,R1,70,62,-101,142.1
2,gen_7_2,67.2,0.0,wind,7,NaN,NaN,0.0,0.0,0.0,0,0.0,R1,2,74,-293,355.4
3,gen_9_3,67.2,0.0,wind,9,NaN,NaN,0.0,0.0,0.0,0,0.0,R1,3,74,-390,355.4
4,gen_10_4,67.2,0.0,wind,10,NaN,NaN,0.0,0.0,0.0,0,0.0,R1,69,116,-104,142.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,gen_106_57,250.0,0.0,hydro,106,10.4,10.4,12.0,12.0,36.0,0,0.0,R3,48,988,-582,142.1
58,gen_110_58,400.0,0.0,nuclear,110,2.8,2.8,2016.0,2016.0,35.0,0,27728.7,R3,50,900,-707,142.1
59,gen_111_59,250.0,0.0,hydro,111,10.4,10.4,12.0,12.0,36.0,0,0.0,R3,51,987,-699,142.1
60,gen_112_60,67.2,0.0,wind,112,NaN,NaN,0.0,0.0,0.0,0,0.0,R1,52,203,-276,142.1


#### There is much more wind power plants

In [20]:
df["type"].value_counts()

wind       49
hydro       6
nuclear     3
solar       3
thermal     1
Name: type, dtype: int64

In [8]:
output_path = os.path.join("..", "example", "custom", "input", "generation", "case118_l2rpn_wcci")
df.to_csv(os.path.join(output_path, "prods_charac.csv"), index=False)

In [9]:
capacity_factor = np.array([30, 95, 15, np.nan, 25])
average_load = 2800
capacity_factor_df = pd.DataFrame(data=capacity_factor, columns=['capacity_factor'], index=types)

grid_path = os.path.join(output_path, "grid.json")
env118_withoutchron = grid2op.make(
    "rte_case118_example",
    test=True,
    grid_path=grid_path, # assign it the 118 grid
    chronics_class=ChangeNothing, # tell it to change nothing (not the most usable environment...)
)

Target_EM_percentage=pd.DataFrame(data=[9, 36, 17, 2, 36], columns=['target_energy_mix'], index=types)

#Variable used to anticipate the energy mix a priori. Update them after chronics generation if too different
PeakLoad = 4200 #expected peak load

EnergyMix_AprioriChecker(env118_withoutchron, Target_EM_percentage, PeakLoad, average_load, capacity_factor_df)

/home/gaetan/.local/lib/python3.8/site-packages/grid2op/MakeEnv/Make.py:305: UserWarning: You are using a development environment. This environment is not intended for training agents. It might not be up to date and its primary use if for tests (hence the "test=True" you passed as argument). Use at your own risk.
  warnings.warn(_MAKE_DEV_ENV_WARN)


,target_energy_mix,pmax,capacity_mix,capacity_factor,Apriori_energy_mix,revised_pmax
type,,,,,,
hydro,9,1500.000000,23.400000,30.0,16.071429,840.000000
nuclear,36,1200.000000,18.700001,95.0,40.714286,1061.052632
solar,17,224.099991,3.500000,15.0,1.200536,3173.333333
thermal,2,200.000000,3.100000,NaN,12.613752,3138.947368
wind,36,3292.799805,51.299999,25.0,29.399998,4032.000000


#### You can see that the difference between the target energy mix and the apriori energy mix is 45%

### We now try to do the OPF

In [14]:
### CONSTANT

notebook_folder=%pwd

# define your input folder
INPUT_FOLDER = os.path.join(notebook_folder, '..', 'example', 'custom', 'input')

OUTPUT_FOLDER = os.path.join(notebook_folder, '..', 'example', 'custom', 'output')

# Detailed configuration to be set in <INPUT_FOLDER>/<CASE>/params.json
start_date = "2012-01-01"
weeks = 8
n_scenarios = 1
by_n_weeks = 4

mode = 'RLTK'


CASE = 'case118_l2rpn_wcci'

In [15]:
cli_chronix2grid="chronix2grid \
                         --mode {} --output-folder {} --input-folder {} --ignore-warnings \
                         --weeks {} --case {} --n_scenarios {} --start-date {} --by-n-weeks {}".format(
                         mode, OUTPUT_FOLDER, INPUT_FOLDER, weeks, CASE, n_scenarios, start_date,by_n_weeks)
cli_chronix2grid

'chronix2grid                          --mode RLTK --output-folder /home/gaetan/Documents/Projects/ChroniX2Grid/getting_started/experiments/../example/custom/output --input-folder /home/gaetan/Documents/Projects/ChroniX2Grid/getting_started/experiments/../example/custom/input --ignore-warnings                          --weeks 8 --case case118_l2rpn_wcci --n_scenarios 1 --start-date 2012-01-01 --by-n-weeks 4'

#### But it doesn't converge

In [16]:
!$cli_chronix2grid

case118_l2rpn_wcci
initial_seeds
{'loads': 1249388886, 'renewables': 1249388886, 'dispatch': 1249388886}
seeds for scenario: Scenario_0
{'loads': 1249388886, 'renewables': 1249388886, 'dispatch': 1249388886}
scenario_path: /home/gaetan/Documents/Projects/ChroniX2Grid/getting_started/experiments/../example/custom/output/generation/case118_l2rpn_wcci/2012-01-01/Scenario_0
============================================== CHRONICS GENERATION ==================================================================
================ Generating Scenario_0 ================
Computing global auto-correlated spatio-temporal noise for thermosensible demand...
Computing loads ...
Saving files in zipped csv in "/home/gaetan/Documents/Projects/ChroniX2Grid/getting_started/experiments/../example/custom/output/generation/case118_l2rpn_wcci/2012-01-01/Scenario_0"
Computing global auto-correlated spatio-temporal noise for sun and wind...
Generating solar and wind production chronics
Saving files in zipped csv
mod